In [ ]:
#! /usr/bin/python

In [ ]:
#previous line makes this an executable python file. Execution line = ./UNIPROT_data_mining_simplified_cluster.py

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re as re
import io

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
#import data
import numpy as np 
import pandas as pd
import csv


In [ ]:
#Load the Uniprot (genefamilies) or Metacyc (pathways) dataframe produced from HumanN2 

path_data = pd.read_csv("D:/all_genefamilies_unstratified.csv")
print("Uniprot IDs found in metagenomes loaded")
path_data.head(5)

In [ ]:
#subset of only five lines of the orginal DF for code testing before running entire dataframe
path_data = path_data.head(50)

#Clean up the ID for generating the URL in a new column
path_data['URL'] = path_data['ID'].str.split(':').str[0].str.replace('UniRef50_','')

path_data.head(5)


In [ ]:
#if the file doesn't have a column for the URL already you can use this to make a new column in the dataframe

#for the metacyc pathways 
#path_data['URL.link'] = 'https://biocyc.org/META/NEW-IMAGE?object=' + path_data['ID'].astype(str) 
#path_data.head(10)

#for the uniprot 
path_data['URL.link'] = 'https://www.uniprot.org/uniprot/' + path_data['URL'].astype(str) + '.txt?version=3'
path_data['URL.link'].head(10)

In [ ]:
# remove Unmapped and unintigrated rows
path_data = path_data[~path_data.ID.str.contains("UNMAPPED")]
path_data = path_data[~path_data.ID.str.contains("UNITEGRATED")]

path_data.head(5)

In [ ]:
#test the URL status to avoid 404 http error
import requests 
from multiprocessing.dummy import Pool as ThreadPool 

urls = path_data['URL.link']

def url_access(x):
    for url in path_data['URL.link']:
        response= requests.get(url)
        status= response.status_code
        path_data['Status']=status

        return requests.head(x).status_code



#use this to run prior code in parallel with multiple threads    
if __name__ == "__main__":
    pool = ThreadPool(4)  # Make the Pool of workers (specific number of threads you want to use)
    results = pool.map(url_access, urls) #Open the urls in their own threads
    pool.close() #close the pool and wait for the work to finish 
    pool.join() 
    

In [ ]:
path_data['Status'].value_counts()
#we are looking for nothing in the 400 range

In [ ]:
#The heavy lifting code that extracts the taxonomic data. 
import requests
import time
from multiprocessing.dummy import Pool as ThreadPool 

startTime = time.time()


#change to different name of the column that has the URLs
urls = path_data['URL.link']#.head(15)
new_df = []

#this def will run a the following commands when the command get_status is called
def get_status(url):
    try:
        req = Request(url)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page, "html.parser")
        html_text = soup.get_text()
        f = open("html_text.txt", "w",encoding='utf-8')
        for line in html_text:
            f.write(line)
        f.close()

        #html_text = html_text.replace("\n", "") # this was needed for the metacyc not the Uniprot
        #return html_text #This if not uncommented out will return the html page, this is a control

        #pattern = re.findall('{}(.*?){}'.format('Range:', 'Synonyms:'), html_text) # metacyc
        pattern = re.findall('{}(.*?){}'.format('OC\s+', '\n'), html_text) #uniprot, Do not replace teh \n with nothing
        #print(pattern)
        #print(len(pattern))
        if len(pattern)==0:# alternate search for pathway data. These if statements find the taxa from metacyc.org
            pattern = re.findall('{}(.*?){}'.format('Range:', 'BioCyc ID:'), html_text)
        #    print(pattern)
            if len(pattern)==0:
                pattern = re.findall('{}(.*?){}'.format('Range:', 'Note'),html_text)  
        new_df.append(pattern) 
        return pattern # this will return the string of interest. 
    except:
        return'Error Invalid URL bypassing, overriding error'
        new_df.append(x)
        pass
if __name__ == "__main__":
    pool = ThreadPool(4)  # Make the Pool of workers (specific number of threads in this case 2)
    results = pool.map(get_status, urls) #Open the urls in their own threads
    pool.close() #close the pool and wait for the work to finish 
    pool.join() 
    
print('Completed multithreaded taxonomic data mininng:\n')
#print(results) # this is the result of the html collection
#print(new_df)
print(len(new_df))

executionTime = (time.time() - startTime)
print('\nExecution time in seconds: ' + str(executionTime)) 

In [ ]:
#Verify output
print(results)
print(len(new_df))
path_data.head(15)

In [ ]:
#pandas function: adds the new_df of the taxa into the existing df while creating new column
df = path_data.assign(taxonomy=new_df)
#df

In [ ]:
#This makes a new column that converts the list/array data generated previouly into a str which i can then reogranize
df['taxonomy'] = df['taxonomy'].astype(str)
df

In [ ]:
m = df['taxonomy'].str.contains('Bacteria|Archaea')

#df1 now contains everything but Bacteria and Archaea
df1 = df[~m].reset_index(drop=True) 
df1

In [ ]:
#this is will produce the df that has Bacteria and Archaea
df2 = df[m].reset_index(drop=True)
df2

In [ ]:
df['taxonomy.str'].value_counts()

In [ ]:
#save the dataframe to a location with all the taxonomy data
#You can partition the taxa data first either in python or Notepad++ to convert the ";" --> "," for csv 
#you would then have to make new labels for each taxanomic rank
df1.to_csv("/home/FCAM/jgil/Beth_files/non_bacterial_mallard_path_abund_taxa.csv")
df2.to_csv("/home/FCAM/jgil/Beth_files/bacterial_mallard_path_abund_taxa.csv")